In [1]:
import json
import pandas as pd
from unidecode import unidecode
import re

with open('data/subset_epita.json') as f:
    data = json.load(f)
    
def get_docs_from_id(id):
    docs = []
    for o in data:
        if o['document_type_id'] == id:
            docs.append(o)
    return docs

datas = get_docs_from_id(8)

def preprocess(word):
    lower = str.lower(word)
    normalized = unidecode(lower)
    punc = '''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ '''
    for ele in normalized:
        if ele in punc:
            normalized = normalized.replace(ele, "")
    return normalized

# Récupérer les mots par paragraphe

In [62]:
def abs(a, b):
    r = a - b
    if r < 0:
        return -r
    return r

def regrouper_mots_par_hauteur(doc, tolerance):
    mots_regroupes = []
    
    keys = list(doc.keys())
    keys.pop()
    keys.pop()
    for key in keys:
        page = {}
        lines = doc[key]
        for line in lines:
            for w in line['line']['words']:
                hauteur = w['top']
                found = False
                for h in page:
                    if abs(h, hauteur) <= tolerance:
                        found = True
                        page[h].append(w['text'])
                        break
                if found == True:
                    continue
                if hauteur not in page:
                    page[hauteur] = []
                page[hauteur].append(w['text'])
        mots_regroupes.append(page)
    return mots_regroupes

tolerance = 0.02
docs = []

for doc in datas:
    resultat = regrouper_mots_par_hauteur(doc, tolerance)
    docs.append(resultat)

# Afficher les groupes de mots
doc = docs[29]
for page in doc:
    print("@@@@@@@@@@@@@@@@")
    for hauteur, mots in page.items():
        print(mots)

@@@@@@@@@@@@@@@@
['Laboratoire', 'BIOMAG', '-', 'BEAUMONT', 'BIOMAG', 'BEAUMONT', 'Autorisation', '950032482']
['1', 'rue', 'Louis', 'Blanc', '95260', 'BEAUMONT', 'SUR', 'OISE', 'Téléphone', ':', '01.30.28.68.20', 'Fax', ':', '01.39.37.78.13']
['labo.beaumont@biogroup.f', 'Site', 'internet', ':', 'www.biogroup.fr']
['BIOGROUP', 'biologie', 'médicale']
['Mme', 'Germain', 'Manon', 'Née', 'Germain', 'SAE', 'TANG', 'STEHANE']
['Date', 'de', 'naissance:', '22-08-1940', '(81', 'ans)', 'Sexe:', 'F', 'Dr', '25,', 'rue', 'de', '153', 'AVENUE', 'D', 'ITALIE']
['75013', '-', 'PARIS', 'CABINET', 'IPSO', 'ITALIE', 'Prescrit', 'par', 'le', 'Dr', 'SAE', 'TANG', 'STEHANE', 'CABINET', 'IPSO', 'ITALIE', '75013', 'PARIS', '13', '75013', 'PARIS', '13', '07:20']
['Sauf', 'mention', 'contraire,', 'prélevé', 'le', ':08-02-2022', 'Enregistré', 'le', '08-02-2022', '10:18']
['Edité', 'le', 'Jeudi', '10', 'Février', '2022', 'à', '14:50', '/', 'Compte-rendu', 'complet', 'Référence', '12202080083']
['Intervalle', 

# Récupérer le top des unités trouvées dans les documents

In [3]:
with open("utils/results/bioloinc_units.json", 'r') as fichier_json:
    units = json.load(fichier_json)

In [4]:
units

{'%  réactivité (avec DTT)': ['%'],
 '%  réactivité (sans DTT)': ['%'],
 '% CD10': ['%'],
 '% CD16+CD56+': ['%'],
 '% CD19+': ['%'],
 '% CD3': ['%'],
 '% CD3+CD4+CD8': ['%'],
 '% CD3-CD16+CD56+': ['%'],
 '% CD3-CD19+': ['%'],
 '% CD4': ['%'],
 '% CD45': ['%'],
 '% CD5': ['%'],
 '% CD8': ['%'],
 "% d'hématies foetales": ['%', '%{RBCs}'],
 "% d'hémoglobine foetale": ["% d'Hb", '%{Hb}'],
 '% de réactivité IgG anti-HLA classe 1': ['%'],
 '% de réactivité IgG anti-HLA classe 2': ['%'],
 '1,25 dihydroxy vitamines D': ['pmol/L', 'pg/mL'],
 '1-hydroxymidazolam': ['ng/mL'],
 '1-méthylhistidine': ['µmol/L', 'umol/L'],
 '1-méthylhistidine/créatinine': ['µmol/mmol créat', 'umol/mmol{creat}'],
 '11-déoxycortisol': ['nmol/L', 'ng/mL'],
 '11-déoxycortisol J01 après métopirone': ['nmol/L'],
 '11-déoxycortisol avant métopirone': ['nmol/L'],
 '17-hydroxyprogestérone': ['nmol/L', 'ng/mL'],
 '17-hydroxyprogestérone salivaire': ['nmol/L'],
 '17-hydroxyprégnénolone': ['nmol/L', 'ng/mL'],
 '2-aminoadipate': 

In [59]:
# Récupérer la liste des marqueurs avec le nombre d'occurences
top_120_df = pd.read_csv('utils/results/top_120.csv')

# Garder que les marqueurs qui sont apparus au moins une fois
top_50_df = top_120_df.drop('Unnamed: 0', axis=1)[:50]
top_50_list = list(map(lambda x : preprocess(x), list(top_50_df['grandeur'])))
print(top_50_list)

# Modifier le dictionnaire des unités pour garder que ceux qui apparaissent
all_units = set()
units_found = {}
for key in units :
    normalized_key = preprocess(key)
    unit_lower = []
    for u in units[key]:
        all_units.add(u.lower())
        unit_lower += u.lower()
        
    if normalized_key in top_50_list:
        units_found[normalized_key] = unit_lower

all_units.add('t/l')
all_units.add('g/100ml')
all_units.add('/mm3')
all_units.add('giga/l')
all_units.add('u3')
all_units.add('/mm3')
all_units.add('picog.')
all_units.add('meg/l')
all_units.add('meq/l')
all_units.add('neg/ml')
all_units.add('ng/nl')

['leucocytes', 'hematies', 'lymphocytes', 'hematocrite', 'hemoglobine', 'monocytes', 'cellules', 'creatinine', 'plaquettes', 'cholesterol', 'triglycerides', 'titre', 'tsh', 'leucocyturie', 'alat', 'ferritine', 'asat', 'potassium', 'sodium', 'proteines', 'cylindres', 'vgm', 'ccmh', 'bilirubine', 'uree', 'chlore', 'crp', 'lh', 'hcg', 'albumine', 'erythrocytes', 'volume', 'erythrocytes', 'citrate', 'calcium', 'tcmh', 'prolactine', 'gentamicine', 'folates', 'cefixime', 'amoxicilline', 'piperacilline', 'amikacine', 'fsh', 'ofloxacine', 'microalbuminurie', 'testosterone', 'transferrine', 'ticarcilline', 'progesterone']


# Itérer sur les paragraphes pour trouver les marqueurs

In [60]:
def search_marqueur(words, start):
    for i in range(start, len(words)):
        normalized_word = preprocess(words[i])
        if normalized_word in units_found.keys():
            return i, normalized_word
    return -1, None

def search_unit(words, start, marqueur):
    for i in range(start + 1, len(words)):
            if words[i].lower() in units_found[marqueur]:
                return i, words[i]
    return -1, None    

def search_other_unit(words, start, marqueur):
    for i in range(start + 1, len(words)):
        if words[i].lower() in all_units :
            units_found[marqueur] += words[i]
            unit_in_doc = words[i]
            index_unit = i
            return words[i], i
    return None, -1

def search_value(words, start, end):
    for i in range(start + 1, end):
        j = i
        res = ""
        while j < end and bool(re.match("^[<>]?[0-9., ]+$", words[j])) == True:
            res += words[j]
            j+=1
        if res != "" :
            return res
        elif words[i] in ['<', '>']:
            return words[i]+words[i+1]
    # Si la valeur n'est pas entre le marqueur et l'unité, on vérifie si la valeur est après l'unité
    if bool(re.match("^[<>]?[0-9., ]+$", words[end + 1])) == True:
        return words[end + 1]
    else :
        return None
    
def is_literal_value(word):
    literal = ['positive', 'negative', 'positif', 'negatif']
    preprocessed_word = preprocess(word)
    if preprocessed_word in literal :
        return preprocessed_word
    return None

def extract_values(doc):
    """
        doc : list of dict
    """
    values_found = {}
    for page in doc :
        for words in page.values():
            # Etape 1 : trouver le marqueur
            index_marqueur, marqueur = search_marqueur(words, 0)
            while index_marqueur != -1 :
                # Etape 1.5 : vérifier si la valeur est "Positive" ou "Négative"
                """value = is_literal_value(words[index_marqueur + 1])
                if value != None :
                    values_found[marqueur] = value
                    break"""
            
                # Etape 2 : trouver l'unité avec bioloinc
                index_unit, unit_in_doc = search_unit(words, index_marqueur, marqueur)
                

                # Etape 3 : si l'unité ne correspond pas alors on va chercher dans la liste des unités
                if index_unit == -1 :
                    unit_in_doc, index_unit = search_other_unit(words, index_marqueur, marqueur)
                    if index_unit == -1 :
                        break
                        
                # Etape 4 : récupérer la valeur qui est entre le marqueur et l'unité
                value = search_value(words, index_marqueur, index_unit)
                if value == None:
                    break
                        
                # Etape 5 : si l'unité est % alors on vérifie s'il y a le mot 'soit' ou ':' juste après
                if unit_in_doc == '%' and (words[index_unit + 1] == 'soit' or words[index_unit + 1] == ':'):
                    index_unit, unit_in_doc = search_unit(words, index_unit+2, marqueur)
                    value = search_value(words, index_marqueur, index_unit)
                    if value == None or index_unit == -1 :
                        break

                # Etape 6 : stocker dans un dictionnaire
                # Si le marqueur est dans les 50 marqueurs 
                if marqueur in top_50_list :
                    values_found[marqueur] = value.replace(" ", "") + unit_in_doc
                
                # On relance pour vérifier s'il y a d'autres marqueurs sur la même ligne
                index_marqueur, marqueur = search_marqueur(words, index_unit + 1)

            
    return values_found


In [61]:
for i in range(len(docs)) :
    print("Document index", i)
    print(extract_values(docs[i]))
    print('------------------')

Document index 0
{'leucocytes': '5000/mL', 'hematies': '<1000/mL'}
------------------
Document index 1
{}
------------------
Document index 2
{}
------------------
Document index 3
{'creatinine': '70,8umol/L'}
------------------
Document index 4
{'albumine': '55,50%', 'hematies': '4,71T/L', 'hemoglobine': '13,5g/dL', 'hematocrite': '42,2%', 'vgm': '90fL', 'tcmh': '28,7pg', 'ccmh': '32,0g/dL', 'leucocytes': '6400/mm3', 'plaquettes': '264000/mm3', 'crp': '<0,6mg/L', 'uree': '0,22g/L', 'creatinine': '89mL/min', 'sodium': '136mmol/L', 'potassium': '4,4mmol/L', 'calcium': '92mg/L', 'ferritine': '22ng/mL', 'titre': '0,51', 'tsh': '0,98mUI/L'}
------------------
Document index 5
{'leucocytes': '5,1G/L', 'hemoglobine': '12,5%', 'volume': '93fL', 'tcmh': '31,3pg', 'ccmh': '33,5%', 'lymphocytes': '30,6%', 'monocytes': '9,8%', 'plaquettes': '222G/L'}
------------------
Document index 6
{}
------------------
Document index 7
{}
------------------
Document index 8
{}
------------------
Document ind